In [1]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )    

In [2]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

MODEL_ID = "LoftQ/Mistral-7B-v0.1-4bit-64rank"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, 
    torch_dtype=torch.bfloat16,  # you may change it with different models
    attn_implementation="flash_attention_2",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,  # bfloat16 is recommended
        bnb_4bit_use_double_quant=False,
        bnb_4bit_quant_type='nf4',
    ),
    device_map={"":0}
)
base_model.resize_token_embeddings(len(tokenizer))

peft_model = PeftModel.from_pretrained(
    base_model,
    MODEL_ID,
    subfolder="loftq_init",
    is_trainable=True,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
from datamodule import datamodule

#path = ["/home/elicer/M-LLM/data/BoolQA.csv", "/home/elicer/M-LLM/data/NLI_CB.csv", "/home/elicer/M-LLM/data/sc_amazon.csv"]
path = "/home/elicer/M-LLM/data/BoolQA.csv"

train_dataset, val_dataset, test_dataset = datamodule.preprare_dataset(path)
# train_dataset = train_dataset.map(lambda samples: tokenizer(samples["text"]), batched=True)
# val_dataset = val_dataset.map(lambda samples: tokenizer(samples["text"]), batched=True)
# test_dataset = test_dataset.map(lambda samples: tokenizer(samples["text"]), batched=True)

Parameter 'function'=<function preprocessing_BoolQA.<locals>.<lambda> at 0x7faebf1f2b90> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2000 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

In [4]:
from datasets import Dataset

text_data = {'text': train_dataset['text']}
train_dataset = Dataset.from_dict(text_data)
train_dataset = train_dataset.map(lambda samples: tokenizer(samples["text"]), batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [5]:
import wandb
import os

wandb.login()
os.environ["WANDB_PROJECT"]="M-LLM"

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rion_. Use `wandb login --relogin` to force relogin


True

In [9]:
import transformers

args = transformers.TrainingArguments(
    num_train_epochs = 1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    fp16=True,
    logging_steps=10,
    output_dir="outputs",
    optim="sgd",
    report_to="wandb",
    run_name="Mistral-BoolQA",
    lr_scheduler_type="cosine",
)

trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    args=args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
peft_model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()
wandb.finish()

Step,Training Loss
10,2.088300
20,2.272000
30,2.069700
40,2.148900
50,2.249800
60,2.052300
70,2.125900
80,2.113200
90,2.035700
100,2.149300


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,███████▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train/loss,▄█▄▅▃▅▅▃▇▁▇▄▆▅▆▄▃▆▃▆▅▆▃▄▆▄▅▇▅▄▅▄▃▇▅▄▆▅▅▅
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,1.0
train/global_step,500


In [11]:
peft_model.push_to_hub("JD97/BoolQA")

adapter_model.safetensors:   0%|          | 0.00/671M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/JD97/BoolQA/commit/bf9d1facb2ece0a13276581f84d8e564f9cbe7cd', commit_message='Upload model', commit_description='', oid='bf9d1facb2ece0a13276581f84d8e564f9cbe7cd', pr_url=None, pr_revision=None, pr_num=None)

In [1]:
merged_model = peft_model.merge_and_unload()

merged_model.save_pretrained(merged_model)
tokenizer.save_pretrained(merged_model)
merged_model.push_to_hub(merged_model)

NameError: name 'peft_model' is not defined

In [14]:
merged_model.push_to_hub("JD97/BoolQA_merged")

NotImplementedError: You are calling `save_pretrained` on a 4-bit converted model. This is currently not supported

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
from vllm import LLM, SamplingParams
from vllm.model_executor.adapters import lora

# Create an LLM.
llm = LLM(model="facebook/opt-125m", gpu_memory_utilization=0.05)

# Add LoRA adapter
lora.LoRAModel.from_pretrained(llm.llm_engine.workers[0].model, "edbeeching/opt-125m-imdb-lora")

prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = SamplingParams(temperature=0, top_k=-1)

outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

In [2]:
from vllm.model_executor.adapters import lora


ModuleNotFoundError: No module named 'vllm.model_executor.adapters'

In [6]:
!git clone --branch support_peft https://github.com/troph-team/vllm.git
!cd vllm
!pip install -e . --user

Cloning into 'vllm'...
remote: Enumerating objects: 5026, done.
remote: Total 5026 (delta 0), reused 0 (delta 0), pack-reused 5026
Receiving objects: 100% (5026/5026), 3.94 MiB | 38.11 MiB/s, done.
Resolving deltas: 100% (3526/3526), done.
Obtaining file:///home/elicer/M-LLM
ERROR: file:///home/elicer/M-LLM does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [7]:
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 1.5 MB/s eta 0:00:00:00:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 4.4 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 30.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 51.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89989 sha256=8005e679304f45913e6ba555d1ff6e03fdc08896de761cb6045c0cc55c2e01ae
  Stored in directory: /home/elicer/.cache/pip/wheels/27/2c/b6/3ed2983b1b44fe0dea1bb35234b09f2c22fb8ebb308679c922
Successfully built lit
  Attempting uninstall: triton
    Found existing installation: triton 2.1.0
    Uninstalling trit